Soccer wizard

In [1]:
import requests
import pandas as pd
import helper_functions as sc

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

In [2]:
# get API key
key = open('data/rapid_api_key.txt', 'r').read()

In [3]:
# get leagues
leagues = sc.request_leagues()
for l in leagues:
    if leagues[l]['country'] == 'Germany':
        print('name = ', leagues[l]['name'], 'year = ', leagues[l]['season'], 'key =', leagues[l]['league_id'])

name =  Bundesliga 1 year =  2018 key = 8
name =  Bundesliga 2 year =  2018 key = 9
name =  Bundesliga 1 year =  2017 key = 35
name =  Bundesliga 2 year =  2017 key = 36
name =  Bundesliga 1 year =  2016 key = 54
name =  Bundesliga 2 year =  2016 key = 55
name =  Liga 3 year =  2018 key = 159
name =  Liga 3 year =  2017 key = 160
name =  Liga 3 year =  2016 key = 161
name =  DFB Pokal year =  2016 key = 202
name =  DFB Pokal year =  2017 key = 203
name =  DFB Pokal year =  2018 key = 204
name =  Women Bundesliga year =  2016 key = 226
name =  Women Bundesliga year =  2017 key = 227
name =  Women Bundesliga year =  2018 key = 228
name =  DFB Pokal year =  2019 key = 583
name =  Bundesliga 1 year =  2015 key = 702
name =  Bundesliga 1 year =  2014 key = 703
name =  Bundesliga 1 year =  2013 key = 704
name =  Bundesliga 1 year =  2012 key = 705
name =  Bundesliga 1 year =  2011 key = 706
name =  Bundesliga 1 year =  2010 key = 707
name =  Liga 3 year =  2019 key = 753
name =  Bundesliga 1

In [4]:
# load data
fixtures = sc.request_data(league=8, fixtures_teams='fixtures')
teams_info = sc.request_data(league=8, fixtures_teams='teams')

In [5]:
# get teams
teams = pd.DataFrame({'id': [], 'team': []})
for num in teams_info:
    teams = teams.append(pd.DataFrame({'id':[teams_info[str(num)]['team_id']], 'team': [teams_info[str(num)]['name']]}))
teams = teams.reset_index(drop=True)
teams.head()

,id,team
0,157,Bayern Munich
1,167,1899 Hoffenheim
2,158,Fortuna Dusseldorf
3,170,FC Augsburg
4,159,Hertha Berlin


In [62]:
# get results
results = pd.DataFrame()
for f in fixtures:
    results = results.append(pd.DataFrame({'round': [fixtures[f]['round']],
                                           'home': [fixtures[f]['homeTeam_id']],
                                           'away': [fixtures[f]['awayTeam_id']],
                                           'goals_home': [fixtures[f]['goalsHomeTeam']],
                                           'goals_away': [fixtures[f]['goalsAwayTeam']],
                                           'match_id': fixtures[f]['fixture_id'],
                                           'match_date': pd.to_datetime(fixtures[f]['event_date'][0:10])}))
results = results.reset_index(drop=True)

# get points
results['points_home'] = results.apply(lambda x: sc.points(x['goals_home'], x['goals_away']), axis=1)
results['points_away'] = results.apply(lambda x: sc.points(x['goals_away'], x['goals_home']), axis=1)
results.head()

,round,home,away,goals_home,goals_away,match_id,match_date,points_home,points_away
0,Regular Season - 1,157,167,3,1,2517,2018-08-24,3,0
1,Regular Season - 1,158,170,1,2,2518,2018-08-25,0,3
2,Regular Season - 1,159,171,1,0,2519,2018-08-25,3,0
3,Regular Season - 1,160,169,0,2,2520,2018-08-25,0,3
4,Regular Season - 1,161,174,2,1,2521,2018-08-25,3,0


In [11]:
#results = pd.read_csv('data\\results_2017.csv', index_col=0)
teams = pd.read_csv('data/teams_2018.csv', index_col=0, sep=',')
teams['id'] = teams.apply(lambda x: str(x['id']) , axis = 1)
n_gameday = len(results['round'].unique())

In [63]:
results = (results.merge(teams.add_prefix('home_'), left_on='home', right_on='home_id', how='left')
                  .merge(teams.add_prefix('away_'), left_on='away', right_on='away_id', how='left')
                  .drop(['home_id', 'away_id'], axis=1))
results.head()

,round,home,away,goals_home,goals_away,match_id,match_date,points_home,points_away,home_team,home_team_web,home_team_odds,away_team,away_team_web,away_team_odds
0,Regular Season - 1,157,167,3,1,2517,2018-08-24,3,0,Bayern Munich,bayern,Bayern Munich,1899 Hoffenheim,hoffenheim,Hoffenheim
1,Regular Season - 1,158,170,1,2,2518,2018-08-25,0,3,Fortuna Dusseldorf,duesseldorf,Fortuna Dusseldorf,FC Augsburg,augsburg,Augsburg
2,Regular Season - 1,159,171,1,0,2519,2018-08-25,3,0,Hertha Berlin,herthabsc,Hertha,FC Nurnberg,nuernberg,Nurnberg
3,Regular Season - 1,160,169,0,2,2520,2018-08-25,0,3,SC Freiburg,freiburg,Freiburg,Eintracht Frankfurt,frankfurt,Ein Frankfurt
4,Regular Season - 1,161,174,2,1,2521,2018-08-25,3,0,VfL Wolfsburg,wolfsburg,Wolfsburg,FC Schalke 04,schalke,Schalke 04


In [64]:
# get rid of 'relegation'
results = results[results['round'] != 'Finals']

In [14]:
# get match statistics WEBSCRAPER
stats = {}

for r in results['round'].unique():
    stats[r] = {}

for i in range(len(results)):
    gameday = results['round'][i][17:]
    # get url
    home = results['home_team_web'][i]
    away = results['away_team_web'][i]
    url = 'https://www.fussballdaten.de/bundesliga/2019/'+gameday+'/'+home+'-'+away+'/'
    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    divs = soup.find_all('div')
    status = divs[1].find_all('div', {'class': "statistik-reihe"})
    clean = re.compile('<.*?>')
    # get team id
    home = results['home'][i]
    away = results['away'][i]
    gameday = results['round'][i]
    # initialize dictionary
    stats[gameday][home] = {}
    stats[gameday][away] = {}    
    for row in status:
        column_name = row.find('div', {'class': "text-center"})
        column_name = (re.sub(clean, '', str(column_name)))
        values = row.find_all('span', {'class':'circle-statistik'})
        values = [(re.sub(clean, '', str(value))) for value in values]
        if len(values) == 0:
            values += ['00,0', '00,0']
            #print(t, r, column_name, 'value is not available')
        stats[gameday][home][column_name] = values[0]
        stats[gameday][away][column_name] = values[1]

In [15]:
rounds = results['round'].unique()
table = {r:{t:{} for t in results['home'].unique()} for r in rounds}

In [16]:
# setup table with base results
last_round = ''
for r in rounds:
    for t in results['home'].unique():
        try:
            match_id = results[(results['round'] == r) & (results['home'] == t)]['match_id']
            points = int(results[(results['round'] == r) & (results['home'] == t)]['points_home'])
            goals_shot = int(results[(results['round'] == r) & (results['home'] == t)]['goals_home'])
            goals_received = int(results[(results['round'] == r) & (results['home'] == t)]['goals_away'])
        except:
            match_id = int(results[(results['round'] == r) & (results['away'] == t)]['match_id'])
            points = int(results[(results['round'] == r) & (results['away'] == t)]['points_away'])
            goals_shot = int(results[(results['round'] == r) & (results['away'] == t)]['goals_away'])
            goals_received = int(results[(results['round'] == r) & (results['away'] == t)]['goals_home'])
        table[r][t]['points'] = points
        if last_round == '':
            table[r][t]['points_cum'] = points
            table[r][t]['goal_difference'] = goals_shot-goals_received
        else:
            table[r][t]['points_cum'] = table[last_round][t]['points_cum'] + points
            table[r][t]['goal_difference'] = (table[last_round][t]['goal_difference'] + goals_shot - goals_received)
    last_round = r
#table

In [17]:
# get rank & rating
for r in rounds:
    gameday = pd.DataFrame()
    for t in results['home'].unique():
        gameday = gameday.append(pd.DataFrame({'round': [r],
                                               'team': [t],
                                               'points_cum': table[r][t]['points_cum'],
                                               'goal_difference': table[r][t]['goal_difference']}))
    gameday = gameday.sort_values(['points_cum', 'goal_difference'], ascending = False)
    gameday['rank'] = [i for i in range(1,len(gameday)+1)]
    for t in results['home'].unique():
        table[r][t]['rank'] = int(gameday[gameday['team'] == t]['rank'])
        table[r][t]['rating'] = sc.get_rating(int(table[r][t]['rank']))

In [18]:
# get form
for t in results['home'].unique():
    form = []
    form_rating = []
    for r in rounds:
        if len(form) > 4:
            form.pop(0)
            form_rating.pop(0)
        form.append(table[r][t]['points'])
        form_rating.append(table[r][t]['rating'])
        table[r][t]['form'] = form.copy()
        table[r][t]['form_rating'] = form_rating.copy()

In [19]:
# get match statistics
for t in results['home'].unique():
    form_posession = []
    form_duels = []
    form_passes = []
    form_accuracy = []
    form_offense = []
    for r in rounds:
        if len(form_posession) > 4:
            form_posession.pop(0)
            form_duels.pop(0)
            form_passes.pop(0)
            form_accuracy.pop(0)
            form_offense.pop(0)
        form_posession.append(int(stats[r][t]['Ballbesitz (%)'][:2]))
        form_duels.append(int(stats[r][t]['Zweikämpfe (%)'][:2]))
        form_passes.append(int(stats[r][t]['Pässe (%)'][:2]))
        accuracy = round(int(stats[r][t]['Tore'])/(int(stats[r][t]['Schüsse aufs Tor'])+0.00001),2)
        form_accuracy.append(accuracy)
        form_offense.append(int(stats[r][t]['Tore']))
        table[r][t]['form_possession'] = form_posession.copy()
        table[r][t]['form_duels'] = form_duels.copy()
        table[r][t]['form_passes'] = form_passes.copy()
        table[r][t]['form_accuracy'] = form_accuracy.copy()
        table[r][t]['form_offense'] = form_offense.copy()

# get odds data

In [87]:
# prepare odds data
data_odds = pd.read_csv('data/betting_odds_2018.csv')
data_odds = data_odds[['HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A', 'Date']]

data_odds = data_odds.merge(teams[['id', 'team_odds']], how='left', 
                            left_on='HomeTeam',
                            right_on='team_odds')
data_odds = data_odds.merge(teams[['id', 'team_odds']], how='left',
                            left_on='AwayTeam',
                            right_on='team_odds')
data_odds = data_odds[['B365H', 'B365D', 'B365A', 'Date', 'id_x', 'id_y']]
data_odds.head()

,B365H,B365D,B365A,Date,id_x,id_y
0,1.20,7.50,12.00,24/08/2018,157,167
1,2.37,3.25,3.00,25/08/2018,158,170
2,2.30,3.30,3.25,25/08/2018,160,169
3,2.04,3.40,3.75,25/08/2018,159,171
4,2.80,3.50,2.45,25/08/2018,163,168


In [89]:
results['match_date'] = pd.to_datetime(results['match_date']).dt.strftime('%m/%d/%Y')
data_odds['Date'] = pd.to_datetime(data_odds['Date'], dayfirst=True).dt.strftime('%m/%d/%Y')

In [93]:
results = results.merge(data_odds, how='left', left_on=['home', 'away', 'match_date'], right_on=['id_x', 'id_y', 'Date'])
results = results[['round', 'home', 'away', 'goals_home', 'goals_away', 'match_id',
       'match_date', 'points_home', 'points_away', 'home_team',
       'B365H', 'B365D', 'B365A']]
results.head()

,round,home,away,goals_home,goals_away,match_id,match_date,points_home,points_away,home_team,B365H,B365D,B365A
0,Regular Season - 1,157,167,3,1,2517,08/24/2018,3,0,Bayern Munich,1.20,7.50,12.00
1,Regular Season - 1,158,170,1,2,2518,08/25/2018,0,3,Fortuna Dusseldorf,2.37,3.25,3.00
2,Regular Season - 1,159,171,1,0,2519,08/25/2018,3,0,Hertha Berlin,2.04,3.40,3.75
3,Regular Season - 1,160,169,0,2,2520,08/25/2018,0,3,SC Freiburg,2.30,3.30,3.25
4,Regular Season - 1,161,174,2,1,2521,08/25/2018,3,0,VfL Wolfsburg,3.10,3.30,2.37


## Get dataframe with features+target:

In [94]:
def get_sample_row(gameday, last_gameday, team_1, team_2, results, i, table, stats, home_away):
    form_ratings_1 = sc.get_rating_value(table[last_gameday][team_1]['form_rating'])
    form_ratings_2 = sc.get_rating_value(table[last_gameday][team_2]['form_rating'])
    form_weighted_1 = sum([table[last_gameday][team_1]['form'][i]*form_ratings_1[i] for i in range(len(form_ratings_1))])
    form_weighted_2 = sum([table[last_gameday][team_2]['form'][i]*form_ratings_2[i] for i in range(len(form_ratings_2))])
    if form_weighted_1 > 0:
        form_weighted_1 = form_weighted_1/len(form_ratings_1)
    if form_weighted_2 > 0:
        form_weighted_2 = form_weighted_1/len(form_ratings_2)
    sample = pd.DataFrame({
        'round': gameday,
        'team_1': team_1,
        'goal_difference_1': table[last_gameday][team_1]['goal_difference'],
        'rating_1': table[last_gameday][team_1]['rating'],
        'form_1': sum(table[last_gameday][team_1]['form'])/len(table[last_gameday][team_1]['form']),
        'form_weighted_1': form_weighted_1, 
        'form_possession_1': sum(table[last_gameday][team_1]['form_possession'])/len(table[last_gameday][team_1]['form_possession']),
        'form_pass_acc_1': sum(table[last_gameday][team_1]['form_passes'])/len(table[last_gameday][team_1]['form_passes']),   
        'form_shot_acc_1': sum(table[last_gameday][team_1]['form_accuracy'])/len(table[last_gameday][team_1]['form_accuracy']),
        'form_duels_1': sum(table[last_gameday][team_1]['form_duels'])/len(table[last_gameday][team_1]['form_duels']),
        'form_offense_1': sum(table[last_gameday][team_1]['form_offense'])/len(table[last_gameday][team_1]['form_offense']),
        'team_2': team_2,
        'goal_difference_2': table[last_gameday][team_2]['goal_difference'],
        'rating_2': table[last_gameday][team_2]['rating'],
        'form_2': sum(table[last_gameday][team_2]['form'])/len(table[last_gameday][team_2]['form']),
        'form_weighted_2' : form_weighted_2,
        'form_possession_2': sum(table[last_gameday][team_2]['form_possession'])/len(table[last_gameday][team_2]['form_possession']),
        'form_pass_acc_2': sum(table[last_gameday][team_2]['form_passes'])/len(table[last_gameday][team_2]['form_passes']),      
        'form_shot_acc_2': sum(table[last_gameday][team_2]['form_accuracy'])/len(table[last_gameday][team_2]['form_accuracy']),
        'form_duels_2': sum(table[last_gameday][team_2]['form_duels'])/len(table[last_gameday][team_2]['form_duels']),        
        'form_offense_2': sum(table[last_gameday][team_2]['form_offense'])/len(table[last_gameday][team_2]['form_offense']),
        'home_away': [home_away],
        'match_id': [results.loc[i]['match_id']],
        'target': [results.loc[i]['points_home']],
        'target_possession': stats[last_gameday][team_1]['Pässe (%)'][:2],
        'match_date': [results.loc[i]['match_date']],
        'B365H': [results.loc[i]['B365H']],
        'B365D': [results.loc[i]['B365D']],
        'B365A': [results.loc[i]['B365A']],})
    return sample

In [95]:
data = pd.DataFrame()
for i in range(len(results)):
    team_1 = results.loc[i]['home']
    team_2 = results.loc[i]['away']
    gameday = results.loc[i]['round']
    if gameday in ['Bundesliga - 1', 'Regular Season - 1']:
        last_gameday = gameday
        continue
    data = data.append(get_sample_row(gameday, last_gameday, team_1, team_2, results, i, table, stats, 1))
    data = data.append(get_sample_row(gameday, last_gameday, team_2, team_1, results, i, table, stats, 0))
    last_gameday = gameday
data = data.reset_index(drop=True)
data.head()

,round,team_1,goal_difference_1,rating_1,form_1,form_weighted_1,form_possession_1,form_pass_acc_1,form_shot_acc_1,form_duels_1,...,form_duels_2,form_offense_2,home_away,match_id,target,target_possession,match_date,B365H,B365D,B365A
0,Regular Season - 2,166,0,C,1.0,2.0,48.0,81.0,0.250,47.0,...,42.0,4.0,1,2526,1,81,08/31/2018,5.50,4.33,1.57
1,Regular Season - 2,165,3,A,3.0,12.0,49.0,78.0,0.800,42.0,...,47.0,1.0,0,2526,1,78,08/31/2018,5.50,4.33,1.57
2,Regular Season - 2,167,0,C,1.5,3.0,48.0,78.0,0.315,45.0,...,52.0,0.5,1,2527,3,80,09/01/2018,1.55,4.00,6.50
3,Regular Season - 2,160,-4,D,0.0,0.0,45.0,75.5,0.085,52.0,...,45.0,2.0,0,2527,3,74,09/01/2018,1.55,4.00,6.50
4,Regular Season - 2,168,-4,D,0.0,0.0,50.5,81.5,0.165,52.5,...,48.5,2.5,1,2528,0,78,09/01/2018,1.61,4.33,5.00


In [22]:
#team_1 = pd.get_dummies(data['team_1'], prefix = 'team_1_', drop_first = True)
#team_2 = pd.get_dummies(data['team_2'], prefix = 'team_2_', drop_first = True)
rating_1 = pd.get_dummies(data['rating_1'], prefix = 'team_1', drop_first = True)
rating_2 = pd.get_dummies(data['rating_2'], prefix = 'team_2', drop_first = True)

In [96]:
data_ready = (data.join(rating_1)
                  .join(rating_2)
                  .drop(['rating_1', 'rating_2'], axis=1))
data_ready.head()

,round,team_1,goal_difference_1,form_1,form_weighted_1,form_possession_1,form_pass_acc_1,form_shot_acc_1,form_duels_1,form_offense_1,...,match_date,B365H,B365D,B365A,team_1_B,team_1_C,team_1_D,team_2_B,team_2_C,team_2_D
0,Regular Season - 2,166,0,1.0,2.0,48.0,81.0,0.250,47.0,1.0,...,08/31/2018,5.50,4.33,1.57,0,1,0,0,0,0
1,Regular Season - 2,165,3,3.0,12.0,49.0,78.0,0.800,42.0,4.0,...,08/31/2018,5.50,4.33,1.57,0,0,0,0,1,0
2,Regular Season - 2,167,0,1.5,3.0,48.0,78.0,0.315,45.0,2.0,...,09/01/2018,1.55,4.00,6.50,0,1,0,0,0,1
3,Regular Season - 2,160,-4,0.0,0.0,45.0,75.5,0.085,52.0,0.5,...,09/01/2018,1.55,4.00,6.50,0,0,1,0,1,0
4,Regular Season - 2,168,-4,0.0,0.0,50.5,81.5,0.165,52.5,0.5,...,09/01/2018,1.61,4.33,5.00,0,0,1,0,0,0


## save csv files

In [97]:
data_ready.to_csv('data/bundesliga_2018_ready.csv')
#data.to_csv('data\\bundesliga_2018_full.csv')

## save dictionary

In [25]:
#f = open("data\\bundesliga_2018_dict.txt","w")
#f.write( str(table) )
#f.close()